In [99]:
# suatu sistem rekomendasi dapat dikatakan memiliki akurasi yang tinggi apabila semakin banyak GT item yang dapat dengan tepat direkomendasikan pada rangking teratas

In [100]:
# import 

import pandas as pd 
import numpy as np 
import math as math
import joblib
from joblib import dump
import os
import time

## PRECISION

$$ Precision(Gt_u, TopN_u, N) = \frac{\left| TopN_u \cap GT_u \right|}{\left| TopN_u(N) \right|} $$
$$ TopN_u(N) = N, sehingga $$
$$ Precision(Gt_u, TopN_u, N) = \frac{\left| TopN_u \cap GT_u \right|}{\left| N \right|} $$

In [101]:
# toy data 
gt_user1 = [1, 2, 3, 4, 11, 12]  # ground truth relevan
topn_user1 = [1, 5, 2, 3, 15, 4]  # hasil rekomendasi sistem


def Precision(GroundTruth, TopN, N):
    rumusPrecision = len(np.intersect1d(TopN[:N], GroundTruth)) / N
    print(len(np.intersect1d(TopN[:N], GroundTruth)))
    print((np.intersect1d(TopN[:N], GroundTruth)))
    # print(rumusPrecision)
    return rumusPrecision

In [102]:
calPrecsionMatrik = Precision(gt_user1, topn_user1,3)
print(calPrecsionMatrik)

2
[1 2]
0.6666666666666666


## RECALL


$$ Recall(GT_u, TopN_u, N) = \frac{\left| TopN_u \cap GT_u \right|}{\left| GT(N) \right|} $$

In [103]:

def Recall(GT, TopN, N):
    rumusRecall = len(np.intersect1d(TopN[:N], GT)) / len(set(GT))
    print(len(np.intersect1d(TopN[:N], GT)))
    print(len(set(GT)))
    # print(rumusRecall)
    return rumusRecall

In [104]:
calRecalMatriks = Recall(gt_user1, topn_user1, 3)
calRecalMatriks


2
6


0.3333333333333333

## F1 SCORE

$$ f1Score(GT_u, TopN_u, N) = \frac{\left| 2 * Precision(Gt_u, TopN_u, N) * Recall(Gt_u, TopN_u, N)  \right|}{\left| Precision(Gt_u, TopN_u, N) + Recall(Gt_u, TopN_u, N)  \right|} $$

In [105]:
def F1Score(GT, TopN, N):
    precision = Precision(GT, TopN, N)
    recall = Recall(GT, TopN, N)
    rumusF1Score = ((2 * precision * recall) / (precision + recall)) 
    if precision > 0 and recall > 0: 
        rumusF1Score = rumusF1Score
    else :
        rumusF1Score = 0
    print(rumusF1Score)
    return rumusF1Score
    

In [106]:
calF1Score = F1Score(gt_user1, topn_user1, 4)

3
[1 2 3]
3
6
0.6


## AP(Average Precision)


$$
AP(GT_u, TopN_u, N) = \frac{1}{|GT_u|} \sum_{n=1}^{N} Precision(GT_u, TopN_u, N)  \cdot \parallel (TopN_u(n) \in  GT_u) $$



In [107]:
def AP(groundTruth, TopN, N):
    aPrecision = np.array([Precision(groundTruth, TopN, N=x) for x in range(1, N+1)])
    print(f'ini adalah preceison {aPrecision}')
    cekGT = np.array([1 if tp in groundTruth else 0 for tp in TopN[:N]])
    print(f'ini cek gt {cekGT}')
    rumusAP = (1/len(groundTruth) * np.sum(aPrecision * cekGT))
    print(np.sum(aPrecision * cekGT))
    print((len(groundTruth)))
    return rumusAP

In [108]:
calAPMatriks = AP(gt_user1, topn_user1, 4)
calAPMatriks

1
[1]
1
[1]
2
[1 2]
3
[1 2 3]
ini adalah preceison [1.         0.5        0.66666667 0.75      ]
ini cek gt [1 0 1 1]
2.4166666666666665
6


0.40277777777777773

## MSE

$$ MSE = \frac{\sum_{(u,i)\in GT^{e^{2}}ui}^{}}{\left| GT \right|} $$

$$ dimana, e = r_{ui} - \hat{r_{ui}} $$

In [109]:
## toy data
gt = [4, 3, 4, 5, 3, 2]
pred = [5, 4, 3, 2, 1, 1]


In [110]:
def MSE(groundTruth, ratingPrediksi):
    # Convert lists to NumPy arrays
    groundTruth_np = np.array(groundTruth)
    ratingPrediksi_np = np.array(ratingPrediksi)
    
    rumusMSE = np.mean((groundTruth_np - ratingPrediksi_np) ** 2)
    print(groundTruth_np)
    print(ratingPrediksi_np)
    print((groundTruth_np - ratingPrediksi_np))
    print((groundTruth_np - ratingPrediksi_np)** 2)
    print(np.sum((groundTruth_np - ratingPrediksi_np)** 2))
    print(len(groundTruth))
    return rumusMSE

In [111]:
calMSE = MSE(gt, pred)
calMSE


[4 3 4 5 3 2]
[5 4 3 2 1 1]
[-1 -1  1  3  2  1]
[1 1 1 9 4 1]
17
6


2.8333333333333335

## RMSE

$$ RMSE = \sqrt{\frac{\sum_{(u,i)\in GT^{e^{2}}ui}^{}}{\left| GT \right|}} $$


$$ dimana, e = r_{ui} - \hat{r_{ui}} $$

In [112]:
def RMSE (groundTruth, ratingPrediksi):
    rumusRMSE = np.sqrt(MSE(groundTruth, ratingPrediksi))
    return rumusRMSE

In [113]:
calrMSE = RMSE(gt, pred)
print(calrMSE)

[4 3 4 5 3 2]
[5 4 3 2 1 1]
[-1 -1  1  3  2  1]
[1 1 1 9 4 1]
17
6
1.6832508230603465


## MAE


$$ MAE = \frac{\sum_{(u,i) \in  E}^{}\left| e_{ui} \right|}{\left| E \right|} $$

In [120]:
def MAE (groundTruth, ratingPrediksi):
    groundTruth_np = np.array(groundTruth)
    ratingPrediksi_np = np.array(ratingPrediksi)
    rumusMAE = np.mean(np.abs(groundTruth_np - ratingPrediksi_np))
    print(groundTruth_np)
    print(ratingPrediksi_np)
    print((groundTruth_np - ratingPrediksi_np))
    print(np.abs((groundTruth_np - ratingPrediksi_np)))
    print(np.sum((np.abs(groundTruth_np - ratingPrediksi_np))))
    print(len(groundTruth))
    return rumusMAE

In [121]:
calMAE = MAE(gt, pred)
print(calMAE)

[4 3 4 5 3 2]
[5 4 3 2 1 1]
[-1 -1  1  3  2  1]
[1 1 1 3 2 1]
9
6
1.5


## NRMSE

$$ NRMSE = \frac {RMSE}{r_{max} - r_{min}} $$

In [116]:
def NRMSE(groundTruth, ratingPrediksi):
    groundTruth_np = np.array(groundTruth)
    ratingPrediksi_np = np.array(ratingPrediksi)
    rumusNRMSE = RMSE(groundTruth_np, ratingPrediksi_np) / (groundTruth_np.max() - groundTruth_np.min())
    return rumusNRMSE

In [117]:
calNRMSE = NRMSE(gt, pred)
print(calNRMSE)

[4 3 4 5 3 2]
[5 4 3 2 1 1]
[-1 -1  1  3  2  1]
[1 1 1 9 4 1]
17
6
0.5610836076867821


## NMAE

$$ NMAE = \frac {MAE}{r_{max} - r_{min}} $$ 

In [118]:
def NMAE(groundTruth, ratingPrediksi):
    groundTruth_np = np.array(groundTruth)
    ratingPrediksi_np = np.array(ratingPrediksi)
    rumusNMAE = MAE(groundTruth_np, ratingPrediksi_np) / (groundTruth_np.max() - groundTruth_np.min())
    return rumusNMAE

In [119]:
calNMAE = NMAE(gt, pred)
print(calNMAE)

0.5
